In [ ]:
#!pip install pandasql

In [1]:
import pandas as pd 
import numpy as np
import os
import math
import collections
import filecmp
from lxml import etree as ET
#import xml.etree.ElementTree as ET
import random
import datetime
from collections import OrderedDict
#import pandasql as ps
from operator import itemgetter
import csv

In [6]:
a = np.zeros((3,5))
a.reshape(a.shape[1], 3)

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [35]:
def check1(a=True):
    if a==True:
        print(1)
    else:
        print(0)

check1(True)
check1(a=False)
check1(a=True)

1
0
1


In [40]:
#path = 'G:/My Drive/2020/sumo/Transit_data_KCM/google_transit'
path = 'G:/Shared drives/SUMO Seattle Simulation Model/Seattle Network/Bigger Seattle/Traffic signal/'
os.chdir(path)
#os.listdir()

In [41]:
col_list = ['sumo_id', 'synchro_id']
id_transfer = pd.read_csv('id_transfer.csv', header=0, usecols=col_list)
id_transfer.head(3)

,sumo_id,synchro_id
0,gneJ355,141.0
1,53073086,164.0
2,53147418,165.0


In [6]:
#get the traffic signal id
tree = ET.parse('Seattle02262021_veh_ped.net.xml')
root = tree.getroot()
signalid_info = []
for signalinfo in root.findall('tlLogic'):
    signalid_info.append(signalinfo.get('id'))

'''with open('signal_id.csv','w',newline='') as f:
    write = csv.writer(f)
    write.writerow(['sumo_id'])
    for i in signalid_info:
        write.writerow([i])'''

"with open('signal_id.csv','w',newline='') as f:\n    write = csv.writer(f)\n    write.writerow(['sumo_id'])\n    for i in signalid_info:\n        write.writerow([i])"

In [42]:
sumo_to_synchro = {}
for i in np.arange(len(id_transfer['sumo_id'])):
    sumo_to_synchro[id_transfer['sumo_id'][i]] = id_transfer['synchro_id'][i]

In [43]:
def write_corresponding_key(sumoid, synchroid, transferdict, revise=False):
    if sumoid in list(transferdict.keys()):
        print('warning, repeated junction, check before revised')
        if revise == False:
            return(transferdict)
        else:
            transferdict[str(sumoid)] = str(synchroid)
            return(transferdict)    
    else:
        transferdict[str(sumoid)] = str(synchroid)
    return(transferdict)    

In [60]:
sumo = 53089007
synchro = 673
sumo_to_synchro = write_corresponding_key(sumo, synchro,sumo_to_synchro)

In [73]:
transfer_sig = pd.DataFrame.from_dict(sumo_to_synchro, orient='index')
transfer_sig.reset_index(inplace=True)

In [74]:
transfer_sig.columns = ['sumoid','synchroid']

In [88]:
transfer_sig['sumoid'] = transfer_sig['sumoid'].astype(str)
transfer_sig['synchroid'] = transfer_sig['synchroid'].astype(float)

In [94]:
transfer_sig.head(3)

,sumoid,synchroid
0,gneJ355,141.0
1,53073086,164.0
2,53147418,165.0


In [97]:
transfer_sig.to_csv(path+'transfer_sigid.csv', index=False)

In [4]:
#scan the addtional file for bus stops
#before
#with open('busstops.add.xml') as file:
#    for line in (file.readlines() [0:100]): 
#        print(line)

#now
#since we combine the taz.add and busstops.add as 'Taz_bigger_Seattle_all_transit.add.xml'
#this part is to check the 'taz' file instead
with open('Taz_bigger_Seattle_all_transit.add.xml') as file:
    for line in (file.readlines() [300:400]): 
        print(line)

    <busStop id="844" lane="332223485#1_1" startPos="40.13" endPos="60.13"/>

    <busStop id="860" lane="-484863624#1_1" startPos="46.00" endPos="66.00"/>

    <busStop id="875" lane="22927953#1.66_1" startPos="2.38" endPos="22.38" friendlyPos="1"/>

    <busStop id="880" lane="512250031#2_1" startPos="14.01" endPos="34.01" friendlyPos="1"/>

    <busStop id="900" lane="22927963#5_1" startPos="6.90" endPos="26.90" friendlyPos="1"/>

    <busStop id="905" lane="22927964#5_1" startPos="2.00" endPos="20.34" friendlyPos="1"/>

    <busStop id="9300" lane="428247746#1_1" startPos="3.09" endPos="23.09" friendlyPos="1"/>

    <busStop id="940" lane="428087144#2_1" startPos="6.44" endPos="26.44" friendlyPos="1"/>

    <busStop id="950" lane="428087142#2_1" startPos="10.31" endPos="30.31" friendlyPos="1"/>

    <busStop id="970" lane="98243676#0_1" startPos="25.14" endPos="45.14" friendlyPos="1"/>

    <busStop id="98105" lane="465797210#2_1" startPos="0.00" endPos="20.00" friendlyPos="1"/>

 

In [5]:
#get the bus id, lane, pos and length
#convert 
def add_info(stop_info, lane_len):
    info = []
    info.append(int(stop_info.get('id')))
    info.append(stop_info.get('lane'))
    start_pos = float(busstop.get('startPos'))
    end_pos = float(busstop.get('endPos'))
    info.append(start_pos)
    info.append(end_pos)
    info.append(lane_len)
    return(info)
  
tree = ET.parse('Taz_bigger_Seattle_all_transit.add.xml')
root = tree.getroot()
bus_info = []
for busstop in root.findall('busStop'):
    start_pos = float(busstop.get('startPos'))
    end_pos = float(busstop.get('endPos'))
    lane_len = round(np.abs(end_pos-start_pos),0)
    if lane_len == 20:
        bus_info.append(add_info(busstop, lane_len))
    elif lane_len == 30:
        bus_info.append(add_info(busstop, lane_len))
    else:
        bus_info.append(add_info(busstop, lane_len))
        print('warning',busstop.get('id'), lane_len)

#convert to data frame
bus_info = pd.DataFrame(bus_info, columns=['id','lane','startPos','endPos','length'])

warning 10210 11.0
warning 10240 17.0
warning 11033 15.0
warning 11095 17.0
warning 1300 14.0
warning 13285 14.0
warning 1500 19.0
warning 19410 16.0
warning 19460 18.0
warning 19480 17.0
warning 20830 19.0
warning 2262 12.0
warning 29270 14.0
warning 3033 13.0
warning 30670 17.0
warning 3151 19.0
warning 3155 19.0
warning 41965 19.0
warning 41970 12.0
warning 450 19.0
warning 620 16.0
warning 905 18.0
warning 996 17.0


In [6]:
bus_info[bus_info['id']==27420]

,id,lane,startPos,endPos,length
144,27420,-425646004#4_1,48.42,68.42,20.0


In [7]:
#read route bus info csva
route_info = pd.read_csv('route_with_stop_order.csv', header=0)
stops_sumo = pd.read_csv('stops4sumo.csv', header=0)
route_info.head(10)
#check missing ids
ids_all = np.unique(route_info['stop_id'])
ids_Seattle = np.unique(stops_sumo['stop_id'])
route_info = route_info[route_info['stop_id'].isin(ids_Seattle)]
del(stops_sumo)
ids_all = np.unique(route_info['stop_id'])

In [8]:
ids_sumo = np.unique(bus_info['id'])
#check bus stop not drawn in current sumo file
missing_stop = []
for i in ids_all:
    if i not in ids_sumo:
        missing_stop.append(i)
        
for j in ids_sumo:
    if j not in ids_all:
        print(j)

In [9]:
'''
#get the link light rail station 
#and remove it from the current list for bus route assignment
link_ids =np.unique(route_info['stop_id'][route_info['route_short_name']=='Link light rail'])

#get the missing id that is not for link stations
id_outbounds = []
for i in missing_stop:
    if i not in link_ids:
        #print(i)
        id_outbounds.append(i)
#through checking the id, it is found that they are all out of
#the boundary in sumo, thus they are not considered as well

#for bus route assignment
#we will consider current bus stop (no link light rail) in sumo
bus_info = bus_info[~bus_info['id'].isin(link_ids)]
ids_sumo = np.unique(bus_info['id'])
#print(np.unique(bus_info['id']))

#check if bus and link light rail has the same station
for busStop in link_ids:
    lines = np.unique(route_info['route_short_name'][route_info['stop_id']==busStop])
    if len(lines) > 1:
        print(lines)'''

"\n#get the link light rail station \n#and remove it from the current list for bus route assignment\nlink_ids =np.unique(route_info['stop_id'][route_info['route_short_name']=='Link light rail'])\n\n#get the missing id that is not for link stations\nid_outbounds = []\nfor i in missing_stop:\n    if i not in link_ids:\n        #print(i)\n        id_outbounds.append(i)\n#through checking the id, it is found that they are all out of\n#the boundary in sumo, thus they are not considered as well\n\n#for bus route assignment\n#we will consider current bus stop (no link light rail) in sumo\nbus_info = bus_info[~bus_info['id'].isin(link_ids)]\nids_sumo = np.unique(bus_info['id'])\n#print(np.unique(bus_info['id']))\n\n#check if bus and link light rail has the same station\nfor busStop in link_ids:\n    lines = np.unique(route_info['route_short_name'][route_info['stop_id']==busStop])\n    if len(lines) > 1:\n        print(lines)"

In [10]:
'''
route_info = route_info.sort_values(by=['route_id', 'stop_order'])
#check if stop id is selected correctly
#route_info.head(10)
for i in np.unique(route_info['route_short_name']):
    trip_num = len(list(np.unique(route_info['trip_id'][route_info['route_short_name']==i])))
    if trip_num>2:
        print(i, trip_num)
#route_info[route_info['route_short_name']=='3'].sort_values(by=['trip_id','stop_order'])
tripID = list(np.unique(route_info['trip_id'][route_info['route_short_name']=="3"]))
#through checking it is found that route id 100173 has 3 trip id
#thus, we delete tripid '49189781'

route_info = route_info[route_info['trip_id']!=tripID[-1]]
for i in np.unique(route_info['route_short_name']):
    trip_num = len(list(np.unique(route_info['trip_id'][route_info['route_short_name']==i])))
    if trip_num>2:
        print(i, trip_num)'''

'\nroute_info = route_info.sort_values(by=[\'route_id\', \'stop_order\'])\n#check if stop id is selected correctly\n#route_info.head(10)\nfor i in np.unique(route_info[\'route_short_name\']):\n    trip_num = len(list(np.unique(route_info[\'trip_id\'][route_info[\'route_short_name\']==i])))\n    if trip_num>2:\n        print(i, trip_num)\n#route_info[route_info[\'route_short_name\']==\'3\'].sort_values(by=[\'trip_id\',\'stop_order\'])\ntripID = list(np.unique(route_info[\'trip_id\'][route_info[\'route_short_name\']=="3"]))\n#through checking it is found that route id 100173 has 3 trip id\n#thus, we delete tripid \'49189781\'\n\nroute_info = route_info[route_info[\'trip_id\']!=tripID[-1]]\nfor i in np.unique(route_info[\'route_short_name\']):\n    trip_num = len(list(np.unique(route_info[\'trip_id\'][route_info[\'route_short_name\']==i])))\n    if trip_num>2:\n        print(i, trip_num)'

In [11]:
#get bus line and route based on the bus id in SUMO
bus_route = route_info[route_info['stop_id'].isin(ids_sumo)]
#link_route = route_info[route_info['stop_id'].isin(link_ids)]

bus_route = bus_route.sort_values(by=['route_short_name','trip_id','stop_order'])
#check if there is missing bus_stop for each bus route
for lines in np.unique(bus_route['route_short_name']):
    tripID = np.unique(bus_route['trip_id'][bus_route['route_short_name']==lines])
    for trips in tripID:
        stopOrder = bus_route['stop_order'][(bus_route['route_short_name']==lines) & (bus_route['trip_id']==trips)]
        if np.max(stopOrder)-np.min(stopOrder)+1 != len(stopOrder):
            print(lines)
            print(trips)
            print(list(stopOrder))

#based on the check, it is found that there are bus stop missing for streetcar
#thus in our case, the First Hill Streetcar is removed
#due to the bus stop 1630 and 26680 is 
bus_route = bus_route[bus_route['route_short_name']!='First Hill Streetcar']
bus_route = bus_route[bus_route['route_short_name']!='South Lake Union Streetcar']

First Hill Streetcar
40803300
[1, 2, 3, 6, 7, 8, 9, 10]
First Hill Streetcar
40803381
[1, 2, 3, 4, 5, 8, 9, 10]


In [12]:
path2 = "G:/My Drive/2020/sumo/Transit_data_KCM/google_transit/"
stop_times = pd.read_csv(path2+'stop_times.txt', header = 0)
trips = pd.read_csv(path2+'trips.txt', header=0)
#based on the calender.txt, we select the service id for bus during weekdays
#including [90868, 89041, 55578]
trips = trips[trips['service_id'].isin([90868, 89041, 55578])]

In [13]:
stop_info = stop_times.merge(trips, left_on='trip_id', right_on='trip_id')
transfer_df = bus_route.merge(stop_info, left_on=['stop_id','route_id'], right_on=['stop_id','route_id'])

#identify the trip with multiple trips
def generateTripMapping(transfer_df):
    query = """
    SELECT trip_id_y, trip_id_x, 
    count(distinct stop_id) cnt
    from transfer_df t 
    group by trip_id_y, trip_id_x 
    order by trip_id_y, trip_id_x"""
    stop_count_per_trip_pair = ps.sqldf(query, locals())

    query="""
    SELECT s1.*
    from stop_count_per_trip_pair s1
    INNER JOIN
    (SELECT trip_id_y, max(cnt) cnt from stop_count_per_trip_pair group by trip_id_y) s2
    on s1.trip_id_y = s2.trip_id_y and s1.cnt = s2.cnt
    ORDER BY trip_id_y, trip_id_x
    """

    trip_pair = ps.sqldf(query, locals())

    validate_query = """SELECT trip_id_y, count(trip_id_x), max(cnt), min(cnt) from trip_pair group by trip_id_y having count(trip_id_x)>1"""
    ps.sqldf(validate_query, locals())

    validate_query = """
    SELECT 'TripPairsAfterJoin',count(*) from stop_count_per_trip_pair
    UNION select 'DuplicateTripPairs', count(*) from (SELECT count(*) from stop_count_per_trip_pair group by trip_id_y having count(trip_id_x)>1) s
    UNION  SELECT 'ValidTripPairs',count(*) from trip_pair"""
    print(ps.sqldf(validate_query, locals()))
    return trip_pair

generateTripMapping(transfer_df).head(10)

  'TripPairsAfterJoin'  count(*)
0   DuplicateTripPairs       557
1   TripPairsAfterJoin      7073
2       ValidTripPairs      6519


,trip_id_y,trip_id_x,cnt
0,35024277,35024277,5
1,35024278,35024277,5
2,35024279,35024277,5
3,35024281,35024277,5
4,35024283,35024277,5
5,35024286,35024277,5
6,35024296,35024277,5
7,35024298,35024277,5
8,35024299,35024277,5
9,35024303,35024277,5


In [14]:
trip_pair = generateTripMapping(transfer_df)
transfer_df = transfer_df.merge(trip_pair, left_on=['trip_id_x', 'trip_id_y'], right_on=['trip_id_x', 'trip_id_y'])
stop_info = transfer_df[['route_id', 'route_short_name', 'stop_name','stop_id', 'stop_order', 'trip_id_y', 'departure_time', 'arrival_time']].sort_values(by=['route_id','trip_id_y','stop_order'])

  'TripPairsAfterJoin'  count(*)
0   DuplicateTripPairs       557
1   TripPairsAfterJoin      7073
2       ValidTripPairs      6519


In [15]:
#check if the stop_order is based on departure time
for i in list(np.unique(stop_info['route_short_name'])):
    route1 = stop_info[stop_info['route_short_name']==i].sort_values(by=['trip_id_y','stop_order'])
    route2 = stop_info[stop_info['route_short_name']==i].sort_values(by=['trip_id_y','departure_time'])
    for j in np.unique(route1['trip_id_y']):
        stops1 = list(route1['stop_id'][route1['trip_id_y']==j])
        stops2 = list(route2['stop_id'][route2['trip_id_y']==j])
        for q in range(len(stops1)):
            if stops1[q]!=stops2[q]:
                print(i, j)
                break

In [16]:
#trips.groupby(['route_id','service_id']).count()[-20:]
#seems like service id=90868 is weekday
#check if the departure time looks good
#stop_info[(stop_info['route_short_name']=='271') & (stop_info['stop_id']==3600)].sort_values(by=['departure_time'])[:20]

In [17]:
'''
#check the ridership data from Erica (https://github.com/metromojo/Route_Dynamics)
path3 = "G:/My Drive/2020/sumo/Transit_data_KCM/"
ridership = pd.read_csv(path3+'Zon183Unsum_from_Erica.csv', header=0)
trip_id_rides = np.unique(ridership['Trip_ID'])
trip_id_bus = np.unique(stop_info['trip_id_y'])
print(trip_id_rides)
print(trip_id_bus)

trip_id_rides = np.unique(ridership['TRANS_LINK_ID'])
trip_id_bus = np.unique(stop_info['route_short_name'])
#print(trip_id_rides)
#print(trip_id_bus)
ct = 0
for i in trip_id_bus:
    if i in trip_id_rides:
        ct = ct+1
print('number of similar trip ids:', ct)
print(len(trip_id_bus), len(trip_id_rides))
#In this case, we are not using the current ridership data, 
#instead, the plan is to generate some pseudo data for sumo network checking
del(ridership)
del(trip_id_rides)
del(trip_id_bus)'''

'\n#check the ridership data from Erica (https://github.com/metromojo/Route_Dynamics)\npath3 = "G:/My Drive/2020/sumo/Transit_data_KCM/"\nridership = pd.read_csv(path3+\'Zon183Unsum_from_Erica.csv\', header=0)\ntrip_id_rides = np.unique(ridership[\'Trip_ID\'])\ntrip_id_bus = np.unique(stop_info[\'trip_id_y\'])\nprint(trip_id_rides)\nprint(trip_id_bus)\n\ntrip_id_rides = np.unique(ridership[\'TRANS_LINK_ID\'])\ntrip_id_bus = np.unique(stop_info[\'route_short_name\'])\n#print(trip_id_rides)\n#print(trip_id_bus)\nct = 0\nfor i in trip_id_bus:\n    if i in trip_id_rides:\n        ct = ct+1\nprint(\'number of similar trip ids:\', ct)\nprint(len(trip_id_bus), len(trip_id_rides))\n#In this case, we are not using the current ridership data, \n#instead, the plan is to generate some pseudo data for sumo network checking\ndel(ridership)\ndel(trip_id_rides)\ndel(trip_id_bus)'

In [18]:
#get the differnt bus stop id
#check current bus, of there is wrong/missing bus id in the current bus file
for i in np.unique(bus_info['id']):
    if i not in list(np.unique(stop_info['stop_id'])):
        print(np.unique(route_info['route_short_name'][route_info['stop_id']==i]))
print("---------")
#check missing bus id 
for i in np.unique(stop_info['stop_id']):
    if i not in list(np.unique(bus_info['id'])):
        print(np.unique(route_info['route_short_name'][route_info['stop_id']==i]))
#noticed all the missing id is for streetcar

['First Hill Streetcar']
['South Lake Union Streetcar']
['First Hill Streetcar']
['First Hill Streetcar']
['First Hill Streetcar']
['First Hill Streetcar']
['First Hill Streetcar']
['South Lake Union Streetcar']
['South Lake Union Streetcar']
['South Lake Union Streetcar']
['First Hill Streetcar']
---------


In [19]:
'''
    <trip id="0" type="bus" depart="6:0:0" color="1,1,0" from="2/0to2/1" to="2/0to2/1">
        <stop busStop="busstop1" until="6:30:00"/>
        <stop busStop="busstop2" until="6:30:00"/>
        <stop busStop="busstop3" until="6:30:00"/>
        <stop busStop="busstop4" until="6:30:00"/>
    </trip>
'''

'\n    <trip id="0" type="bus" depart="6:0:0" color="1,1,0" from="2/0to2/1" to="2/0to2/1">\n        <stop busStop="busstop1" until="6:30:00"/>\n        <stop busStop="busstop2" until="6:30:00"/>\n        <stop busStop="busstop3" until="6:30:00"/>\n        <stop busStop="busstop4" until="6:30:00"/>\n    </trip>\n'

In [20]:
def addline(stops, depart_time):
    line = ''
    i = 0 
    for stop in stops:
        line = line+'\t\t<stop busStop="'+str(stop)+'" until="'+depart_time[i]+'"/>\n'
        i = i+1
    return(line)

def create_startingtime(time, n):
    H = time[:2]
    M = time[3:5]
    S = time[-2:]
    #print(H, M,S)
    if int(S)<n:
        if int(M)==0:
            if int(H) == 0:
                H = '00'
                M = '00'
                S = '00'
            elif int(H)<=10 and int(H)!=0:
                H = '0'+str(int(H)-1)
                M = '59'
                S = str(50+int(S))
            else:
                H = str(int(H)-1)
                M = '59'
                S = str(50+int(S))
            
        else:
            if int(M)<=10:
                M = '0'+str(int(M)-1)
            else:
                M = str(int(M)-1)
            S = str(50+int(S))
        
    else:
        s = int(S)-n
        if s<10:
            S = '0'+str(s)
        else:
            S = str(s)
        
        
    time = H+':'+M+':'+S
    return(time)

def convert_time_into24hr(time):
    time_t = time.split(":")
    H = time_t[0]
    if int(H)>=24:
        if int(H)-24<10:
            H = '0'+str(int(H)-24)
            time_t[0] = H
            time = ':'.join(time_t) 
        else:
            print('warnings, there exist time > 34:00:00', time) 
    return(time)

In [21]:
#conver time that exceeds 24:00:00
convert_time = []
for t in stop_info['departure_time']:
    convert_time.append(convert_time_into24hr(t))
#print(len(convert_time)==len(stop_info['departure_time']))

stop_info['convert_departure_time'] = convert_time
del(convert_time)

H1 = []
for i in stop_info['convert_departure_time']:
    H1.append(int(i[:2]))
stop_info['H'] = H1
del(H1)

H2 = []
for i in stop_info['departure_time']:
    H2.append(int(i[:2]))
stop_info['H2'] = H2
del(H2)

In [22]:
stop_info.head(2)
#stop_info[(stop_info['route_short_name'] == 'D Line') & (stop_info['trip_id_y']==49196129)]

,route_id,route_short_name,stop_name,stop_id,stop_order,trip_id_y,departure_time,arrival_time,convert_departure_time,H,H2
882,100001,1,S Jackson St & 5th Ave S,1530,4,49195113,05:02:00,05:02:00,05:02:00,5,5
883,100001,1,Prefontaine Pl S & Yesler Way,1610,5,49195113,05:03:57,05:03:57,05:03:57,5,5


In [23]:
def writelines(idname, tripID, from_, to_, stops, time):
    depart_t = create_startingtime(time[0], 10)
    if 'Link_light_rail' in idname:
        lines = '\t<trip id="'+idname+'_'+str(tripID)+'" type="rail_urban" depart="'+depart_t+\
        '" from="'+from_[0]+'" to="'+to_[0]+'">\n'
    else:
        lines = '\t<trip id="'+idname+'_'+str(tripID)+'" type="bus" depart="'+depart_t+\
        '" from="'+from_[0]+'" to="'+to_[0]+'">\n'
    stop_line = addline(stops, time)
    lines = lines+stop_line+'\t</trip>\n\n' 
    return(lines, depart_t)

def get_needed_var(df):
    stops = list(df['stop_id'])
    from_ = list(bus_info['lane'][bus_info['id'] == stops[0]])[0].split('_')
    to_ = list(bus_info['lane'][bus_info['id'] == stops[-1]])[0].split('_')
    return(stops, from_, to_)

def check_and_write(route, idname, tripID, write_dict):
    tripdf = route[route['trip_id_y']==tripID]
    old_order = tripdf[['stop_order', 'departure_time']].sort_values(by=['departure_time'])
    new_order = tripdf[['stop_order', 'convert_departure_time']].sort_values(by=['convert_departure_time'])
    #if there is dismatach
    if  list(old_order['stop_order']) != list(new_order['stop_order']):
        #if there is only one time exceeds 24:00:00
        #keep the original order
        if len(list(tripdf['H'][tripdf['H']>=24])) == 1:
            tripdf = tripdf.sort_values(by=['stop_order'])
            time = list(tripdf['departure_time'])
            stops, from_, to_ = get_needed_var(tripdf)
            lines, depart_t = writelines(idname, tripID, from_, to_, stops, time)
            write_dict.append([depart_t,lines])
        else:
        #separate the time between 23:00:00 and 24:00:00
            tripdf = tripdf.sort_values(by=['H', 'convert_departure_time'])
            df1 = tripdf[tripdf['H']<=10]
            tripID1 = str(tripID) + '_0'
            time = list(df1['convert_departure_time'])
            stops, from_, to_ = get_needed_var(df1)
            lines, depart_t = writelines(idname, tripID1, from_, to_, stops, time)
            write_dict.append([depart_t,lines])
            
            df2 = tripdf[tripdf['H']>=20]
            tripID2 = str(tripID) + '_1'
            time = list(df2['convert_departure_time'])
            stops, from_, to_ = get_needed_var(df2)
            lines, depart_t = writelines(idname, tripID2, from_, to_, stops, time)
            write_dict.append([depart_t,lines])
            
    elif min(np.unique(tripdf['H2']))>=24:
        tripdf = tripdf.sort_values(by=['stop_order'])
        time = list(tripdf['convert_departure_time'])
        stops, from_, to_ = get_needed_var(tripdf)
        lines, depart_t = writelines(idname, tripID, from_, to_, stops, time)
        write_dict.append([depart_t,lines])
            
    else:
        tripdf = tripdf.sort_values(by=['stop_order'])
        time = list(tripdf['departure_time'])
        stops, from_, to_ = get_needed_var(tripdf)
        lines, depart_t = writelines(idname, tripID, from_, to_, stops, time)
        write_dict.append([depart_t,lines])
        
    return(write_dict)
                         

In [24]:
write_dict=[]
for i in np.unique(stop_info['route_short_name']):
    if ' ' in i:
        name_id = i.split(' ')
        idname = ''
        for j in range(len(name_id)):
            if j != len(name_id)-1:
                idname = idname+name_id[j]+"_"
            else:
                idname = idname+name_id[j]
    else:  
        idname = i
    route = stop_info[stop_info['route_short_name']==i].sort_values(by=['trip_id_y','stop_order'])
    for tripID in np.unique(route['trip_id_y']):
        check_and_write(route, idname, tripID, write_dict)

In [25]:
#write_dict = OrderedDict(sorted(write_dict.items()))
write_dict = sorted(write_dict, key=itemgetter(0))

In [26]:
write_dict

[['00:00:00',
  '\t<trip id="10_41979657" type="bus" depart="00:00:00" from="460423550#0" to="623536799">\n\t\t<stop busStop="1085" until="00:00:00"/>\n\t\t<stop busStop="1110" until="00:01:25"/>\n\t\t<stop busStop="1120" until="00:02:00"/>\n\t\t<stop busStop="1180" until="00:05:00"/>\n\t\t<stop busStop="11130" until="00:07:37"/>\n\t\t<stop busStop="11132" until="00:09:00"/>\n\t\t<stop busStop="13460" until="00:11:13"/>\n\t\t<stop busStop="29268" until="00:13:02"/>\n\t\t<stop busStop="29270" until="00:15:00"/>\n\t</trip>\n\n'],
 ['00:00:00',
  '\t<trip id="10_41979730_0" type="bus" depart="00:00:00" from="460423550#0" to="460423550#0">\n\t\t<stop busStop="1085" until="00:00:00"/>\n\t</trip>\n\n'],
 ['00:00:00',
  '\t<trip id="13_49194874" type="bus" depart="00:00:00" from="460421475#1" to="-456673731">\n\t\t<stop busStop="2672" until="00:00:00"/>\n\t\t<stop busStop="2180" until="00:00:57"/>\n\t\t<stop busStop="2200" until="00:02:05"/>\n\t\t<stop busStop="2244" until="00:03:49"/>\n\t\t<

In [27]:
#write the bus route for .rou.xml
filename = 'bus_link_route.rou.xml'
f = open(filename, 'w')
f.write('<?xml version="1.0" encoding="UTF-8"?>\n')
f.write('<routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">\n')
#define bus type
f.write('<vType vClass="bus" accel="0.8" color="red" decel="3.5" guiShape="bus" id="bus" length="12" maxSpeed="70" sigma="0.7" width="2.5"/>\n')
f.write('<vType vClass="rail_urban" id="rail_urban"/>\n')

#write bus trips
#define the trips and let SUMO generate the route automatically

for k in range(len(write_dict)):
    f.write(write_dict[k][1])

f.write('</routes>')

f.close()

In [28]:
#stop_info[(stop_info['route_short_name']=='D Line')&(stop_info['trip_id_y']==49196064)]
#stop_info[(stop_info['route_short_name']=='21')&(stop_info['trip_id_y']==47580428)]

In [29]:
'''
#previous code without time convert
#depart_time
#bus_stop
write_dict={}
for i in np.unique(stop_info['route_short_name']):
    if ' ' in i:
        name_id = i.split(' ')
        idname = ''
        for j in range(len(name_id)):
            if j != len(name_id)-1:
                idname = idname+name_id[j]+"_"
            else:
                idname = idname+name_id[j]
    else:  
        idname = i
    route = stop_info[stop_info['route_short_name']==i].sort_values(by=['trip_id_y','stop_order'])
    for tripID in np.unique(route['trip_id_y']):
        stops = list(route['stop_id'][route['trip_id_y']==tripID])
        time = list(route['departure_time'][route['trip_id_y']==tripID])
        #print(stops)
        #print(time)
        from_ = list(bus_info['lane'][bus_info['id'] == stops[0]])[0].split('_')
        to_ = list(bus_info['lane'][bus_info['id'] == stops[-1]])[0].split('_')
        #print(from_)
        #print(to_)
        #print(i)
        #print(tripID)
        
        depart_t = create_startingtime(time[0], 10)
        lines = '\t<trip id="'+idname+'_'+str(tripID)+'" type="bus" depart="'+depart_t+'" from="'+from_[0]+'" to="'+to_[0]+'">\n'
        #print(lines)
        stop_line = addline(stops, time)
        write_dict[depart_t] = lines+stop_line+'\t</trip>\n\n'   

#sort dict by depart time
write_dict = OrderedDict(sorted(write_dict.items()))'''

'\n#previous code without time convert\n#depart_time\n#bus_stop\nwrite_dict={}\nfor i in np.unique(stop_info[\'route_short_name\']):\n    if \' \' in i:\n        name_id = i.split(\' \')\n        idname = \'\'\n        for j in range(len(name_id)):\n            if j != len(name_id)-1:\n                idname = idname+name_id[j]+"_"\n            else:\n                idname = idname+name_id[j]\n    else:  \n        idname = i\n    route = stop_info[stop_info[\'route_short_name\']==i].sort_values(by=[\'trip_id_y\',\'stop_order\'])\n    for tripID in np.unique(route[\'trip_id_y\']):\n        stops = list(route[\'stop_id\'][route[\'trip_id_y\']==tripID])\n        time = list(route[\'departure_time\'][route[\'trip_id_y\']==tripID])\n        #print(stops)\n        #print(time)\n        from_ = list(bus_info[\'lane\'][bus_info[\'id\'] == stops[0]])[0].split(\'_\')\n        to_ = list(bus_info[\'lane\'][bus_info[\'id\'] == stops[-1]])[0].split(\'_\')\n        #print(from_)\n        #print(to_

In [30]:
#np.unique(stop_info['route_short_name'])

In [31]:
#print(np.unique(route_info['route_short_name'][route_info['stop_id']==996]))
#print(np.unique(route_info['route_short_name'][route_info['stop_id']==810]))
#print(np.unique(route_info['route_short_name'][route_info['stop_id']==99101]))

In [32]:
'''#check data shared by Dan
path = 'G:/My Drive/2020/sumo/Transit_data_KCM/google_transit'
os.chdir(path)

trips = pd.read_csv('trips.txt', header=0)
stop_times = pd.read_csv('stop_times.txt', header=0)
routes = pd.read_csv('routes.txt', header=0)
stops = pd.read_csv('stops.txt',header=0)
print('trips')
print(trips.columns)
print('stop_times')
print(stop_times.columns)
print('routes')
print(routes.columns)
print('stops')
print(stops.columns)

def check_dataset(df1, df2, check_col):
    print(len(df1[check_col]))
    print(len(df2[check_col]))

    print(len(np.unique(df1[check_col])))
    print(len(np.unique(df2[check_col])))

check_dataset(trips, stop_times, 'trip_id')
check_dataset(routes, trips, 'route_id')
check_dataset(stops, stop_times, 'stop_id')'''

"#check data shared by Dan\npath = 'G:/My Drive/2020/sumo/Transit_data_KCM/google_transit'\nos.chdir(path)\n\ntrips = pd.read_csv('trips.txt', header=0)\nstop_times = pd.read_csv('stop_times.txt', header=0)\nroutes = pd.read_csv('routes.txt', header=0)\nstops = pd.read_csv('stops.txt',header=0)\nprint('trips')\nprint(trips.columns)\nprint('stop_times')\nprint(stop_times.columns)\nprint('routes')\nprint(routes.columns)\nprint('stops')\nprint(stops.columns)\n\ndef check_dataset(df1, df2, check_col):\n    print(len(df1[check_col]))\n    print(len(df2[check_col]))\n\n    print(len(np.unique(df1[check_col])))\n    print(len(np.unique(df2[check_col])))\n\ncheck_dataset(trips, stop_times, 'trip_id')\ncheck_dataset(routes, trips, 'route_id')\ncheck_dataset(stops, stop_times, 'stop_id')"

In [2]:
#traffic signal data processing
path = 'G:/Shared drives/SUMO Seattle Simulation Model/Seattle Network/Bigger Seattle/Traffic signal/'
os.chdir(path)
#os.listdir()

In [3]:
#read traffic signal file
South_Seattle_tp = pd.read_csv('SouthSeattle_time_plans.csv',header=2)
print(South_Seattle_tp.head(3))

Dt_Seattle_tp = pd.read_csv('Downtown_Seattle_time_plans.csv', header=2)
print(Dt_Seattle_tp.head(3))

     RECORDNAME  INTID  DATA
0  Control Type      1     0
1  Cycle Length      1   120
2  Lock Timings      1     0
     RECORDNAME  INTID  DATA
0  Control Type    102     3
1  Cycle Length    102    50
2  Lock Timings    102     0


In [4]:
S_control_type = list(np.unique(South_Seattle_tp['DATA'][South_Seattle_tp['RECORDNAME']=='Control Type']))
print(S_control_type)
Dt_control_type = control_type = list(np.unique(Dt_Seattle_tp['DATA'][Dt_Seattle_tp['RECORDNAME']=='Control Type']))
print(Dt_control_type)

[0, 1, 2, 3]
[0, 1, 2, 3]


In [5]:
#through checking from synchro, we get the control type:
control_type = {0: 'pretimed',\
               1:'Actd-Uncrd',\
               2:'Semi Act-Uncrd',\
               3:'Act-Coord'}

In [100]:
#South_Seattle_tp[(South_Seattle_tp['RECORDNAME']=='Control Type')].sort_values(by=['DATA'])

In [6]:
#get list of ids
#check control typea
#for i in Dt_control_type:
#(Dt_Seattle_tp['RECORDNAME']=='Control Type')&
#get id examoples
for i in Dt_control_type:
    print('Control_type: '+str(i))
    print('Node_example:'+str(list(Dt_Seattle_tp['INTID'][(Dt_Seattle_tp['RECORDNAME']=='Control Type')&(Dt_Seattle_tp['DATA']==i)])[0]))

Control_type: 0
Node_example:151
Control_type: 1
Node_example:560
Control_type: 2
Node_example:146
Control_type: 3
Node_example:102


In [7]:
#get phase data
S_Seattle_phase = pd.read_csv('South_Seattle_Phase.csv', header=2)
Dt_Seattle_phase = pd.read_csv('Downtown_Seattle_phase.csv', header=2)
print(S_Seattle_phase.head(3))
print(Dt_Seattle_phase.head(3))

  RECORDNAME  INTID     D1     D2     D3     D4     D5     D6     D7     D8  \
0        BRP      1  111.0  112.0  211.0  212.0  121.0  122.0  221.0  222.0   
1   MinGreen      1    NaN    7.0    NaN    7.0    NaN    NaN    NaN    NaN   
2   MaxGreen      1    NaN   75.5    NaN   35.5    NaN    NaN    NaN    NaN   

      D9    D10    D11    D12    D13    D14    D15    D16  
0  311.0  312.0  411.0  412.0  321.0  322.0  421.0  422.0  
1    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  
2    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  
  RECORDNAME  INTID     D1     D2     D3     D4     D5     D6     D7     D8  \
0        BRP    102  111.0  112.0  211.0  212.0  121.0  122.0  221.0  222.0   
1   MinGreen    102    NaN    7.0    NaN    7.0    NaN    NaN    NaN    NaN   
2   MaxGreen    102    NaN   21.5    NaN   16.0    NaN    NaN    NaN    NaN   

      D9    D10    D11    D12    D13    D14    D15    D16  D20  
0  311.0  312.0  411.0  412.0  321.0  322.0  421.0  422.0  Na

In [10]:
test_path = path+'test/test_single_intersection/'
#test_path = path+'test/'
os.chdir(test_path)
os.listdir()

['test2_single.net.xml',
 'test2_single_intersection.sumocfg',
 'test2_single.rou.xml',
 'Summary.out.xml',
 'test2.rou.zip',
 'Bigger_Seattle_phase.csv',
 'traffic_signal.add.xml',
 'test_2_signals.net.xml',
 'test_2_signals.rou.xml',
 'transfer_sigid.csv',
 'Bigger_Seattle_timeplan.csv',
 'test2.net.xml']

In [25]:
#read the traffic timing plans
timeplans = pd.read_csv('Bigger_Seattle_timeplan.csv', header = 2)
timeplans.head(3)
phases = pd.read_csv('Bigger_Seattle_phase.csv', header=2)
#transfer id from synchro to sumo
transfer_id = pd.read_csv('transfer_sigid.csv', header=0)

In [26]:
tranfer_id.columns

Index(['Unnamed: 0', 'sumoid', 'synchroid'], dtype='object')

In [27]:
transfer_id.head(3)
transfer_id['synchroid'] = transfer_id['synchroid'].astype(float)
transfer_id['synchroid'] = round(transfer_id['synchroid'], 0)


In [28]:
#scan the data contains in each csv file
print(np.unique(phases['RECORDNAME']))
print(np.unique(timeplans['RECORDNAME']))

['ActGreen' 'AllRed' 'BRP' 'DontWalk' 'DualEntry' 'End' 'InhibitMax'
 'LocalStart' 'LocalYield' 'LocalYield170' 'MaxGreen' 'MinGap' 'MinGreen'
 'MinSplit' 'PedCalls' 'Recall' 'Start' 'TimeBeforeReduce' 'TimeToReduce'
 'VehExt' 'Walk' 'Yellow' 'Yield' 'Yield170']
['Control Type' 'Cycle Length' 'Lock Timings' 'Master' 'Node 0' 'Node 1'
 'Node 2' 'Offset' 'Reference Phase' 'Referenced To' 'Yield']


In [137]:
#covert transfer id csv into dict file
'''transfer_dict = {}
for i in range(len(tranfer_id.index)):
    transfer_dict[tranfer_id['sumoid'][i]] = tranfer_id['synchroid'][i]
    
#check values

for signalid in np.unique(timeplans['INTID']):
    if signalid not in tranfer_id['synchroid']:
        print('synchro not in sumo: ', signalid)
for ids in transfer_dict.values():
    if ids not in np.unique(timeplans['INTID']):
        print('sumo not in synchro: ', ids)
        
#print(tranfer_id['sumoid'][tranfer_id['synchroid'] == 6.0])
#delete keys
#transfer_dict.pop('', None)
#sorted(transfer_dict.items(), key=lambda x: x[1])'''
#Based on the findings we identified that there are some signal infomation missing
#when converting data from synchro to csv file

In [30]:
'''transfer_id = pd.DataFrame.from_dict(transfer_dict, orient='index')
transfer_id.reset_index(inplace=True)
transfer_id.columns = ['sumoid', 'synchroid']
transfer_id.to_csv('transfer_sigid.csv')'''

"transfer_id = pd.DataFrame.from_dict(transfer_dict, orient='index')\ntransfer_id.reset_index(inplace=True)\ntransfer_id.columns = ['sumoid', 'synchroid']\ntransfer_id.to_csv('transfer_sigid.csv')"

In [31]:
#since there are missing data when converting synchro into csv
#we divide the signal data into two part
#data included in the current converted csv
#data not included in the current converted csv
available_id = transfer_id[transfer_id['synchroid'].isin(np.unique(timeplans['INTID']))]
missing_id = transfer_id[~transfer_id['synchroid'].isin(np.unique(timeplans['INTID']))]

In [35]:
available_id = available_id[['sumoid', 'synchroid']]
missing_id = missing_id[['sumoid', 'synchroid']]

In [36]:
available_id.sort_values(['synchroid'])

,sumoid,synchroid
276,4273733662,5.0
270,53128668,23.0
271,3153558838,24.0
272,53131121,26.0
273,53082272,30.0
...,...,...
280,2706300474,709.0
277,gneJ374,710.0
281,53073246,712.0
282,53073245,713.0


In [37]:
path

'G:/Shared drives/SUMO Seattle Simulation Model/Seattle Network/Bigger Seattle/Traffic signal/'

In [38]:
#available_id['synchroid']
ids = list(available_id['synchroid'])
timeplans = timeplans[timeplans['INTID'].isin(ids)]

In [39]:
os.listdir(path+'test/')

['test_2.net.xml',
 'test_2.rou.xml',
 'test_2.sumocfg',
 'Summary.out.xml',
 'test_single_intersection',
 'traffic_signal.add.xml',
 'Bigger_Seattle.csv',
 'desktop.ini']

In [ ]:
#data from time plans
# timeplans
#['Control Type' 'Cycle Length' 'Lock Timings' 'Master' 'Node 0' 'Node 1'
# 'Node 2' 'Offset' 'Reference Phase' 'Referenced To' 'Yield']
# phases
#['ActGreen' 'AllRed' 'BRP' 'DontWalk' 'DualEntry' 'End' 'InhibitMax'
# 'LocalStart' 'LocalYield' 'LocalYield170' 'MaxGreen' 'MinGap' 'MinGreen'
# 'MinSplit' 'PedCalls' 'Recall' 'Start' 'TimeBeforeReduce' 'TimeToReduce'
# 'VehExt' 'Walk' 'Yellow' 'Yield' 'Yield170']

In [261]:
timeplans.head(3)

,RECORDNAME,INTID,DATA
0,Control Type,5,3
1,Cycle Length,5,120
2,Lock Timings,5,0


In [40]:
control_type = {0:'static',1:'actuated', 2:'actuated', 3:'actuated'}

In [42]:
#get info
#ctype
control_type[list(timeplans['DATA'][(timeplans['INTID']==ids[0]) & \
                (timeplans['RECORDNAME']=='Control Type')])[0]]
#offset
str(list(timeplans['DATA'][(timeplans['INTID']==ids[0]) & \
                (timeplans['RECORDNAME']=='Offset')])[0])



'6'

In [43]:
control_type[list(timeplans['DATA'][(timeplans['INTID']==ids[0]) & \
                (timeplans['RECORDNAME']=='Control Type')])[0]]

'actuated'

In [44]:
transfer_id[transfer_id['sumoid'] == "1487927522"]

,Unnamed: 0,sumoid,synchroid
257,257,1487927522,292.0


In [45]:
timeplans[timeplans['INTID']==292]

,RECORDNAME,INTID,DATA
1203,Control Type,292,1
1204,Cycle Length,292,105
1205,Lock Timings,292,0
1206,Referenced To,292,0
1207,Reference Phase,292,2
1208,Offset,292,0
1209,Master,292,0
1210,Yield,292,2
1211,Node 0,292,292
1212,Node 1,292,0


In [46]:
control_type[list(timeplans['DATA'][(timeplans['INTID']==5) & \
                (timeplans['RECORDNAME']=='Control Type')])[0]]

'actuated'

In [47]:
#for write the xml more pretty
def indent(elem, level=0):
    i = "\n" + level*"  "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "  "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            indent(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

In [48]:
param_key = ['max-gap', "detector-gap", "passing-time",\
             "vTypes","show-detectors","file", "freq"]
param_value = ["3.0","2.0","2.0","","false","NULL","300" ]

In [49]:
path

'G:/Shared drives/SUMO Seattle Simulation Model/Seattle Network/Bigger Seattle/Traffic signal/'

In [50]:
#read traffic signal 
tree = ET.parse(path+'test/'+'test_2.net.xml')
root = tree.getroot()

for signalinfo in root.iter('tlLogic'):
    signal_id = signalinfo.get('id')
    if signal_id in list(available_id['sumoid']):
        synchroid = list(transfer_id['synchroid'][transfer_id['sumoid']==signal_id])[0]
        types =str(control_type[list(timeplans['DATA'][(timeplans['INTID']==synchroid) & (timeplans['RECORDNAME']=='Control Type')])[0]])
        offsets = str(list(timeplans['DATA'][(timeplans['INTID']==synchroid) & (timeplans['RECORDNAME']=='Offset')])[0])
        signalinfo.set('type', types)
        signalinfo.set('offset', offsets)
        if types == 'actuated':
            param = [ET.SubElement(signalinfo,'param', \
                                 key = param_key[i], \
                                 value = param_value[i])
                                 for i in range(len(param_key))]
            signalinfo.extend(param)
            
indent(root)    
#with open(path+'test/test_single_intersection/'+'test.xml', 'w') as f:
tree.write(path+'test/'+'test_2_traffic_sig_revise.net.xml', \
            encoding="utf-8", \
           xml_declaration=True)


In [330]:
#write the bus route for .rou.xml
filename = 'bus_link_route.rou.xml'
f = open(filename, 'w')
f.write('<?xml version="1.0" encoding="UTF-8"?>\n')
f.write('<routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">\n')
#define bus type
f.write('<vType vClass="bus" accel="0.8" color="red" decel="3.5" guiShape="bus" id="bus" length="12" maxSpeed="70" sigma="0.7" width="2.5"/>\n')
f.write('<vType vClass="rail_urban" id="rail_urban"/>\n')

#write bus trips
#define the trips and let SUMO generate the route automatically

for k in range(len(write_dict)):
    f.write(write_dict[k][1])

f.write('</routes>')

f.close()

<Element 'tlLogic' at 0x000002E8D0A0F130>